In [24]:
import numpy as np
import random as rnd
import scipy.spatial.distance as distance
import math
import scipy.stats as stats
from sklearn.preprocessing import normalize

In [25]:
D = 4; K = 0; Beta = 0.1; delta = 0.01
mus = np.ndarray(D + 1); sigmas = np.ndarray(D+1)
for i in range(D + 1):
    mus[i] = 2 * i / 3 * D
    sigmas[i] = (i % 3 + 2) / D
    
numvectors = 10; lenvectors = 3
X = np.ndarray((numvectors, lenvectors))

for i in range(numvectors):
    index = i % (D + 1)
    
    for j in range(lenvectors):
        X[i,j] = rnd.gauss(mus[index], sigmas[index])
        
    #print(X[i])
    for j in range(lenvectors):
        summed = sum(X[i])
        X[i,j] = X[i,j] / summed
        
    #print(X[i])

In [26]:
sigma = np.std(X)
sigmaInv = sigma ** -1
#print(sigmaInv)
alpha = sigmaInv * np.identity(lenvectors)
covarDet = np.linalg.det(alpha) ** -1
#print(alpha)

alphas = {}
mus = {}
covDets = {}
comProbs = {}
ages = {}
accumulators ={}

alphas[0] = alpha
covDets[0] = covarDet
mus[0] = X[0]
comProbs[0] = 1
ages[0] = 1
accumulators[0] = 1
K = 1

In [27]:
for i in range(len(X)):
    if i == 0:
        continue
    updated = False
    
    for j in range(K):
        if i == j:
            continue
            
        dist = distance.mahalanobis(X[i], mus[j], alphas[j]) ** 2
        print(dist)
        #dist2 = stats.multivariate_normal.logpdf(X[i], mus[j], np.linalg.inv(alpha))
        #dist3 = math.exp(dist2)
        #print(dist2)
        #print(dist3)
        dist2 = stats.multivariate_normal.pdf(X[i], mus[j], np.linalg.inv(alpha))
        print()
        
        if dist < dist2:
            print('call update')
            update(accumulators, ages, alphas, comProbs, covDets, D, j, K, mus, sigmaInv, X[i])
            updated = True
            
    if not updated:
        print('call create')
        K = create(accumulators, ages, alphas, comProbs, covDets, K, mus, sigmaInv, X[i])
    

SyntaxError: invalid syntax (Temp/ipykernel_9116/3171557721.py, line 13)

In [19]:
def create(accumulators, ages, alphas, comProbs, covDets, K, mus, sigmaInv, x):
    accumulators[K] = 1
    ages[K] = 1
    alphas[K] = sigmaInv * np.identity(len(alphas[0]))
    comProb = 1 / sum(accumulators.values())
    covDets = np.linalg.det(alphas[K]) ** -1
    mus[K] = x
    K += 1
    return K


In [22]:
def update(accumulators, ages, alphas, comProbs, covDets, D, j, K, mus, sigmaInv, x):
    dist = distance.mahalanobis(x, mus[j], alphas[j]) ** 2
    pxj = 1 / ((2 * math.pi) ** (D / 2) * math.sqrt(covDets[j])) * math.exp(-0.5 * dist)
    
    pjx = pxj * comProbs[j]
    totProbs = 0
    for i in range(K):
        pxi =  1 / ((2 * math.pi) ** (D / 2) * math.sqrt(covDets[i])) * math.exp(-0.5 * dist)
        totProbs += pxi * comProb[i]
    pjx /= totProbs
    
    ages[j] += 1
    accumulators[j] += pjx
    
    ej = x - mus[j]
    weight = pjx / accumulators[j]
    
    deltaMu = weight * ej
    mus[j] += deltaMu
    
    oldAlpha = alphas[j]
    ejStar = x - mu[j]
    newAlpha = oldAlpha / (1 - weight) - (weight / (1 - weight) ** 2) * np.matmul(np.matmul(oldAlpha, ejStar), np.matmul(ejStar.transpose(), oldAlpha)) \
            / (1 + (weight / (1 - weight)) * np.matmul(np.matmul(ejStar.transpose(), oldAlpha),ejStar))
    alphas[j] = newAlpha +  (np.matmul(np.matmul(newAlpha, mus[j]), np.matmul(mus[j].transpose(), newAlpha))) \
            / (1 - np.matmul(np.matmul(mus[j].transpose(), newAlpha), mus[j]))
    
    comProb[j] = accumulators[j] / sum(accumulators.values())
    
    newCovDet = (1 - weight) ^ D * covDets[j] * (1 + weight/(1 - weight) * np.matmul(ejStar.transpose(), np.matmul(oldAlpha, ejStar)))
    covDets[j] = newCovDet * (1 - np.matmul(deltaMu.transpose(), np.matmul(newAlpha, deltaMu)))
    
    

1